In [1]:
import networkx as nx
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
from rdflib import Dataset
from networkx import Graph as NXGraph
import matplotlib.pyplot as plt
import statistics
import collections
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON
from math import factorial

## KG exploration

In [6]:
dataset = Dataset()
dataset.parse("resources/linked-jazz.trig", format="trig")

# take the relationship graph
lj = list(dataset.graphs())[1]
lj_G = rdflib_to_networkx_digraph(lj)

List predicates in the KG

In [7]:
set(lj.predicates())

{rdflib.term.URIRef('http://linkedjazz.org/ontology/bandLeaderOf'),
 rdflib.term.URIRef('http://linkedjazz.org/ontology/bandmember'),
 rdflib.term.URIRef('http://linkedjazz.org/ontology/inBandTogether'),
 rdflib.term.URIRef('http://linkedjazz.org/ontology/playedTogether'),
 rdflib.term.URIRef('http://linkedjazz.org/ontology/touredWith'),
 rdflib.term.URIRef('http://purl.org/ontology/mo/collaborated_with'),
 rdflib.term.URIRef('http://purl.org/vocab/relationship/acquaintanceOf'),
 rdflib.term.URIRef('http://purl.org/vocab/relationship/friendOf'),
 rdflib.term.URIRef('http://purl.org/vocab/relationship/hasMet'),
 rdflib.term.URIRef('http://purl.org/vocab/relationship/influencedBy'),
 rdflib.term.URIRef('http://purl.org/vocab/relationship/knowsOf'),
 rdflib.term.URIRef('http://purl.org/vocab/relationship/mentorOf')}

Build the weighted graph

In [4]:
WEIGHTS_MAP = {
    rdflib.term.URIRef('http://linkedjazz.org/ontology/inBandTogether'): 0.8,
    rdflib.term.URIRef('http://linkedjazz.org/ontology/playedTogether'): 0.8,
    rdflib.term.URIRef('http://linkedjazz.org/ontology/touredWith'): 0.9,
    rdflib.term.URIRef('http://purl.org/ontology/mo/collaborated_with'): 0.8,
    rdflib.term.URIRef('http://purl.org/vocab/relationship/acquaintanceOf'): 0.6,
    rdflib.term.URIRef('http://purl.org/vocab/relationship/friendOf'): 0.7,
    rdflib.term.URIRef('http://purl.org/vocab/relationship/hasMet'): 0.2,
    rdflib.term.URIRef('http://purl.org/vocab/relationship/knowsOf'): 0.1,
    rdflib.term.URIRef('http://purl.org/vocab/relationship/mentorOf'): 0.9
}

for i, j in lj_G.edges:
    lj_G[i][j]['weight'] = 0
    for s, p, o in lj_G[i][j]['triples']:
        lj_G[i][j]['weight'] += WEIGHTS_MAP.get(p, 0)

Compute the centrality due to the eigenvectors

In [6]:
ec = nx.eigenvector_centrality(lj_G, weight="weight")

Print the top 10 most influential artists. The label is queried directly from the LinkedJazz KG through a SPARQL query.

In [7]:
def get_dbpedia_label(iri):
  sparql = SPARQLWrapper("https://dbpedia.org/sparql")
  sparql.setReturnFormat(JSON)
  sparql.setQuery("""
  SELECT ?label
  WHERE {
    <%s> rdfs:label ?label .
    FILTER(LANGMATCHES(LANG(?label), \"en\"))
  }
  """ % iri)
  
  try:
    ret = sparql.queryAndConvert()
    label = ret["results"]["bindings"][0]["label"]["value"]
  except:
    label = iri
  return label

for k in sorted(ec, key=ec.get, reverse=True)[:10]:
    label = get_dbpedia_label(str(k))
    print(f"{ec[k]} - {label}")

0.37577864573377373 - Sonny Payne
0.3623081289539205 - Eddie "Lockjaw" Davis
0.30656841680720565 - Eva Jessye
0.2786985607338482 - Bill Hughes (musician)
0.26565482897837156 - Louis Armstrong
0.2605886802410345 - Count Basie
0.25519900471608226 - Miles Davis
0.18766970746297484 - Eubie Blake
0.16814097077664972 - Cat Anderson
0.1366814314506891 - Dizzy Gillespie


## Adjacency matrix experiments

In [22]:
def truncated_graph_power_exp(graph: nx.Graph, degree: int) -> np.array:
  """
  Compute the trucated matrix power exponential using Taylor series expansion.

  Args:
      graph (nx.Graph): Input NetworkX grpah.
      degree (int): Tatlor degree of the polynomial

  Returns:
      np.array: Adjacency matrix
  """
  adj = nx.to_numpy_array(graph)
  powadj = adj
  res = adj
  for i in range(2, degree + 1):
      powadj = np.matmul(powadj, adj)
      coeff = 1 / factorial(i)
      res = res + (coeff * powadj)
  return res

In [259]:
lj_G_exp = truncated_graph_power_exp(lj_G, 8)

How much did Miles Davis influence Herbie Hancock? What about Count Basie?

Generally, the influence of Miles Davis should be much higher, since Hancock mention him as one of his main inspirations.

In [260]:
miles_davis_idx = list(lj_G.nodes()).index(rdflib.URIRef("http://dbpedia.org/resource/Miles_Davis"))
count_basie_idx = list(lj_G.nodes()).index(rdflib.URIRef("http://dbpedia.org/resource/Count_Basie"))
herbie_hancock_idx = list(lj_G.nodes()).index(rdflib.URIRef("http://dbpedia.org/resource/Herbie_Hancock"))

In [261]:
cp_row = lj_G_exp[herbie_hancock_idx, :]
print(f"Miles Davis influence in Herbie Hancock: {cp_row[miles_davis_idx]}")

Miles Davis influence in Herbie Hancock: 34.683024250129954


In [262]:
print(f"Count Basie influence in Herbie Hancock: {cp_row[count_basie_idx]}")

Count Basie influence in Herbie Hancock: 26.2808248954695


Most influential artists to Herbie Hancock

In [263]:
for k in np.argsort(cp_row)[::-1][:20]:
  print(f"{cp_row[k]}: {get_dbpedia_label(list(lj_G.nodes)[k])}")

34.683024250129954: Miles Davis
26.2808248954695: Count Basie
25.41997081068155: Sonny Payne
24.46121225573884: Eddie "Lockjaw" Davis
21.305980485252977: Louis Armstrong
20.69794883177902: Eva Jessye
18.816317119799113: Bill Hughes (musician)
16.018443664504467: Cat Anderson
14.617881374063993: Dizzy Gillespie
13.868866142761657: Ron Carter
12.66094439672619: Eubie Blake
12.631132934747523: http://dbpedia.org/resource/Chick_Corea
12.403779793234376: Duke Ellington
11.522236827831103: Harry Carney
9.99959074698884: Louie Bellson
9.746313824069942: Oscar Peterson
9.468581451294641: http://dbpedia.org/resource/Alex_Acu%C3%B1a
9.023738338122024: Don Redman
8.938050235620038: Donald Byrd
8.922633232347719: Gil Evans


## A vector implementation

The idea is to use an `Embedding` layer, which assigns a tensor to an index, to represent the weights of the relations.

If we have $R$ relations, then the input to the network will be the adjacency matrix for each relation. 
With $N$ the number of nodes, the input to the network is a matrix of the form $x^{N \times N \times R}$.

We can then leverage NN models to approximate the weights.

In [10]:
from typing import List
import torch
from torch import nn

INFLUENCED_BY_PRED = rdflib.URIRef("http://purl.org/vocab/relationship/influencedBy")

class SumIRWE(nn.Module):
  def __init__(self, graph: rdflib.Graph, target_pred: rdflib.URIRef, comunicability_degree: float = 8):
    super().__init__()
    self.c_degree = comunicability_degree

    self.relations = list(set(graph.predicates()))
    nx_graph = rdflib_to_networkx_digraph(graph)

    # build the input graph by building several different directed graphs that only contains nodes connected by a specific predicate
    self.input = None
    for rel in self.relations:
      sub_g = nx.DiGraph()
      sub_g.add_nodes_from(nx_graph.nodes)
      sub_g.add_edges_from(graph.subject_objects(rel))

      # 1 x N x N
      sub_g_adj = torch.tensor(nx.to_numpy_array(sub_g)).unsqueeze(0)

      # |R| x N x N
      self.input = sub_g_adj if self.input is None else torch.cat([self.input, sub_g_adj])

    # build the target graph
    target_g = nx.DiGraph()
    target_g.add_nodes_from(nx_graph.nodes)
    target_g.add_edges_from(graph.subject_objects(target_pred))
    self.target = torch.tensor(nx.to_numpy_array(target_g))

    # NN definition
    self.relation_embedding = nn.Embedding(len(self.relations), 1)
    self.loss = nn.CrossEntropyLoss()

  def forward(self):
    # prepare a weight for each relation
    relations_idxs = torch.arange(len(self.relations))
    
    # compute weight of each relation
    relation_weight = self.relation_embedding(relations_idxs)

    # weight input by casting the weights to |R| x 1 x 1
    w_input = self.input * relation_weight.reshape(-1, 1, 1)

    # combine weights (using sum)
    w_input = w_input.sum(axis=0)

    # compute comunicability up to X degree
    initial_graph = w_input
    graph_power = w_input
    for i in range(2, self.c_degree + 1):
      graph_power = torch.mm(graph_power, initial_graph)
      coeff = torch.tensor(1 / factorial(i))
      w_input = w_input + (coeff * graph_power)
    
    # compute cross entropy loss: the problem is a classification problem
    loss = self.loss(w_input, self.target)

    return loss

model = SumIRWE(lj, INFLUENCED_BY_PRED)

In [12]:
from tqdm import trange

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for step in (pbar := trange(10)):
  loss = model()
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  loss = loss.item()
  pbar.set_description(f"Loss: {loss}")

Loss: 2.997246419989877: 100%|██████████| 10/10 [01:29<00:00,  8.98s/it]


## Optimize top-k

In [24]:
def weight_graph(graph, weight_schema):
  # reset weights
  for i, j in graph.edges:
    graph[i][j]['weight'] = 0
  
  for i, j in graph.edges:
    graph[i][j]['weight'] = 0
    for s, p, o in graph[i][j]['triples']:
        graph[i][j]['weight'] += weight_schema.get(p, 0)

In [25]:
INFLUENCED_BY_PRED = rdflib.URIRef("http://purl.org/vocab/relationship/influencedBy")
influenced_by_rel = [(s, o) for s, _ , o in lj.triples((None, INFLUENCED_BY_PRED, None)) ]

In [26]:
wm = dict(zip(model.relations, [x.item() for x in model.relation_embedding.weight.squeeze(1)]))
weight_graph(lj_G, wm)
lj_G_exp = truncated_graph_power_exp(lj_G, 8)

In [27]:
top_pred = list()
for influenced, influencer in influenced_by_rel:
  influenced_idx = list(lj_G.nodes()).index(influenced)
  influenced_row = lj_G_exp[influenced_idx, :]
  top = [list(lj_G.nodes())[i] for i in np.argsort(influenced_row)[::-1][:10]]
  top_pred.append(influencer in top)

In [29]:
np.sum(top_pred) / len(top_pred)

0.25

In [119]:
for k, v in zip(WEIGHTS_MAP.keys(), weights):
  print(k, v)

http://linkedjazz.org/ontology/inBandTogether 0.62169746
http://linkedjazz.org/ontology/playedTogether 0.17829556
http://linkedjazz.org/ontology/touredWith 1.14409202
http://purl.org/ontology/mo/collaborated_with 1.26547381
http://purl.org/vocab/relationship/acquaintanceOf 0.66776842
http://purl.org/vocab/relationship/friendOf 2.25927431
http://purl.org/vocab/relationship/hasMet 1.32576557
http://purl.org/vocab/relationship/knowsOf 1.4899642
http://purl.org/vocab/relationship/mentorOf 1.67660337
